In [ ]:
import os
import os.path as op
import numpy as np
import matplotlib.pyplot as plt
import mne
import mne_nirs
import pandas as pd

from scipy.stats import sem
from scipy import stats
from itertools import compress
from pprint import pprint
from nilearn.plotting import plot_design_matrix

from mne.viz import plot_compare_evokeds
from mne.preprocessing.nirs import (optical_density,
                                    temporal_derivative_distribution_repair)

from mne_nirs.visualisation import plot_glm_surface_projection
from mne_nirs.experimental_design import make_first_level_design_matrix
from mne_nirs.statistics import run_glm
from mne_nirs.channels import (get_long_channels,
                               get_short_channels,
                               picks_pair_to_idx)

In [ ]:
# WAA analysis
def weighted_averaging_analysis(raw_haemo):

    # remove short channels
    raw_haemo = mne_nirs.channels.get_long_channels(raw_haemo)
    
    # divide events 
    events, event_dict = mne.events_from_annotations(raw_haemo,{'Control': 1, 'Noise': 2, 'Speech': 3})

    # time frame for epochs
    tmin, tmax = -5, 15

    # create epochs
    epochs = mne.Epochs(raw_haemo, events, event_id=event_dict,
                        tmin=tmin, tmax=tmax,
                        reject_by_annotation=True,
                        proj=True, baseline=(None, 0), preload=True,
                        detrend=1, verbose=True)
    
    

    # PAIRED T TEST
    # create arrays for each condition + channelname. roll 2 to get source 12 to its ROI
    chnames = np.roll(epochs.copy().ch_names,2)
    carray = np.roll(np.mean(epochs['Control'].copy().crop(4,6)._data,axis=2),2,axis=1)
    narray = np.roll(np.mean(epochs['Noise'].copy().crop(4,6)._data,axis=2),2,axis=1)
    sarray = np.roll(np.mean(epochs['Speech'].copy().crop(4,6)._data,axis=2),2,axis=1)

    maxlen = max([len(carray),len(narray),len(sarray)])

    # replace missing values with mean of array
    while len(carray) < maxlen:
        carray = np.vstack([carray, np.mean(carray,axis=0)])

    while len(narray) < maxlen:
        narray = np.vstack([narray, np.mean(narray,axis=0)])

    while len(sarray) < maxlen:
        sarray = np.vstack([sarray, np.mean(sarray,axis=0)])

    # difference in control : active stimuli
    ncont = narray - carray
    scont = sarray - carray

    # calculate t-values
    m = len(chnames)
    speechp = np.empty(m)
    noisep = np.empty(m)

    for ii in range(0,m):
        speechp[ii] = stats.t.sf(abs(np.mean(scont[:,ii])/sem(scont[:,ii])), maxlen-1)
        noisep[ii] = stats.t.sf(abs(np.mean(ncont[:,ii])/sem(ncont[:,ii])), maxlen-1)


    # FDR CORRECTION
    WAAres = pd.DataFrame()
    WAAres['Channels'] = chnames
    WAAres['Noise'] = noisep
    WAAres['Speech'] = speechp
    
    # rank array and alpha value
    FDRarray = np.array(range(1,m+1))
    FDRalpha = 0.20

    # array containing the comparison value
    FDRarray = FDRalpha * FDRarray / m

    for jj,cond in enumerate(['Noise','Speech']):
        # ranking by smallest p-value
        WAAres = WAAres.sort_values(by=cond, ascending=True)
        # comparing value to array
        WAAres[cond] = WAAres[cond] < FDRarray
        maxpos = WAAres[cond][::-1].idxmax()
        minpos = WAAres[cond][::-1].idxmin()
        if maxpos == minpos:
            continue
        maxloc = WAAres.index.get_loc(maxpos)
        # setting all lower ranks to true
        WAAres[cond].iloc[0:maxloc + 1] = True

    # sorting by channels again
    WAAres = WAAres.sort_index()
    

    left = [[4, 2], [4, 3], [5, 2], [5, 3], [5, 4], [5, 5]] #, [4, 14], [5, 15]
    right = [[10, 9], [10, 10], [10, 11], [10, 12], [11, 11], [11, 12]] #, [10, 18], [11, 19]
    back = [[6, 6], [6, 8], [7, 6], [7, 7], [7, 8], [8, 7], [8, 8], [9, 8]] #, [6, 16], [8, 17]
    front = [[1, 1], [2, 1], [3, 1], [3, 2], [12, 1]] #, [2, 13], [12, 20]

    ROIs = dict(STG=np.concatenate((np.array(picks_pair_to_idx(raw_haemo, left))/2//1,
                np.array(picks_pair_to_idx(raw_haemo, right))/2//1)),
                Occipital=np.array(picks_pair_to_idx(raw_haemo, back))/2//1,
                Frontal=np.array(picks_pair_to_idx(raw_haemo, front))/2//1)

    ROIs['STG'] = ROIs['STG'].astype(int)
    ROIs['Occipital'] = ROIs['Occipital'].astype(int)
    ROIs['Frontal'] = ROIs['Frontal'].astype(int)

    evoked_dict = {'Noise/HbO': epochs['Noise'].average(picks='hbo'),
                'Noise/HbR': epochs['Noise'].average(picks='hbr'),
                'Speech/HbO': epochs['Speech'].average(picks='hbo'),
                'Speech/HbR': epochs['Speech'].average(picks='hbr'),
                'Control/HbO': epochs['Control'].average(picks='hbo'),
                'Control/HbR': epochs['Control'].average(picks='hbr')}  


    for condition in evoked_dict:
        evoked_dict[condition].rename_channels(lambda x: x[:-4])

    condlist=['Control','Noise','Speech']

    #fig, axes = plt.subplots(nrows=len(ROIs), ncols=len(event_dict),figsize=(14, 10))

    WaaRoiGrp = pd.DataFrame()


    for ridx, roi in enumerate(ROIs):
        for cidx, event in enumerate(event_dict): 
            plot_compare_evokeds([evoked_dict[condlist[cidx]+'/HbO'],evoked_dict[condlist[cidx]+'/HbR']], picks=ROIs[roi], combine="mean", title='', 
                axes=axes[ridx,cidx], show=False,colors=['r','b'], ylim=dict(hbo=[-0.1, 0.2],hbr=[-0.1, 0.2]), legend=False, show_sensors=True, ci=0.95)
            
            axes[0,cidx].set_title(f'{event}')
            axes[ridx, 0].set_ylabel(f"{roi}\n μM")
            
            if cidx == 0:
                if ridx ==0: 
                    contvalhbo = evoked_dict[condlist[cidx]+'/HbO'].copy()
                    contvalhbr = evoked_dict[condlist[cidx]+'/HbR'].copy()
                contvalhbo._data = contvalhbo._data + evoked_dict[condlist[cidx]+'/HbO'].copy()._data
                contvalhbr._data = contvalhbr._data + evoked_dict[condlist[cidx]+'/HbR'].copy()._data


            if cidx == 1:
                if ridx ==0: 
                    noisevalhbo = evoked_dict[condlist[cidx]+'/HbO'].copy()
                    noisevalhbr = evoked_dict[condlist[cidx]+'/HbR'].copy()
                noisevalhbo._data = noisevalhbo._data + evoked_dict[condlist[cidx]+'/HbO'].copy()._data
                noisevalhbr._data = noisevalhbr._data + evoked_dict[condlist[cidx]+'/HbR'].copy()._data

            if cidx == 2:
                if ridx ==0: 
                    speechvalhbo = evoked_dict[condlist[cidx]+'/HbO'].copy()
                    speechvalhbr = evoked_dict[condlist[cidx]+'/HbR'].copy()
                speechvalhbo._data = speechvalhbo._data + evoked_dict[condlist[cidx]+'/HbO'].copy()._data
                speechvalhbr._data = speechvalhbr._data + evoked_dict[condlist[cidx]+'/HbR'].copy()._data

    WaaRoiGrp={}
    WaaRoiGrp['ControlHbO'] = contvalhbo
    WaaRoiGrp['ControlHbR'] = contvalhbr
    WaaRoiGrp['NoiseHbO'] = noisevalhbo
    WaaRoiGrp['NoiseHbR'] = noisevalhbr
    WaaRoiGrp['SpeechHbO'] = speechvalhbo
    WaaRoiGrp['SpeechHbR'] = speechvalhbr


    axes[0, 0].legend(["Oxyhaemoglobin", "Deoxyhaemoglobin"],loc=2)



    color_dict = dict(HbO='#AA3377', HbR='b')
    styles_dict = dict(Control=dict(linestyle='dotted'), Noise=dict(linestyle='dashed'))

    mne.viz.plot_compare_evokeds(evoked_dict, combine="mean", ci=0.95,
                                colors=color_dict, styles=styles_dict,show=True)
    
    return WAAres, WaaRoiGrp, ROIs


In [3]:
def GLM_analysis(raw_haemo):

    # removing break annotations
    unwanted = np.nonzero(raw_haemo.annotations.description == 'Xstart')
    raw_haemo.annotations.delete(unwanted)
    unwanted = np.nonzero(raw_haemo.annotations.description == 'Xend')
    raw_haemo.annotations.delete(unwanted)

    short_chs = get_short_channels(raw_haemo)
    raw_haemo = get_long_channels(raw_haemo)


    events, event_dict = mne.events_from_annotations(raw_haemo, verbose=False)
    mne.viz.plot_events(events, event_id=event_dict, sfreq=raw_haemo.info['sfreq'],show=False)


    s = mne_nirs.experimental_design.create_boxcar(raw_haemo)
    """ fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(15, 6))
    plt.plot(raw_haemo.times, s, axes=axes)
    plt.legend(["Control", "Noise", "Speech"], loc="upper right")
    plt.xlabel("Time (s)") """


    raw_stim = raw_haemo.copy()
    raw_stim.annotations.delete(raw_stim.annotations.description == 'Control')


    isis, names = mne_nirs.experimental_design.longest_inter_annotation_interval(raw_stim)



    design_matrix = make_first_level_design_matrix(raw_stim,
                                                drift_model='cosine',
                                                high_pass=1/(2*max(isis)),  # Must be specified per experiment
                                                hrf_model='spm',
                                                stim_dur=5.125)


  

    #design_matrix["ShortHbO"] = np.mean(short_chs.copy().pick(
    #                                    picks="hbo").get_data(), axis=0)

    #design_matrix["ShortHbR"] = np.mean(short_chs.copy().pick(
    #                                    picks="hbr").get_data(), axis=0)


    """ fig, ax1 = plt.subplots(figsize=(10, 6), nrows=1, ncols=1)
    fig = plot_design_matrix(design_matrix, ax=ax1) """


    glm_est = run_glm(raw_haemo, design_matrix, noise_model='auto')


    subjects_dir = op.join(mne.datasets.sample.data_path(), 'subjects')
    mne.utils.set_config("SUBJECTS_DIR", subjects_dir, set_env=True)
    

    GLMresN = pd.DataFrame()
    GLMresS = pd.DataFrame()
    df2 = glm_est.to_dataframe()


    GLMresN['ch_name'] = df2['ch_name'][df2.Condition == 'Noise']
    GLMresN['Noise'] = df2['p_value'][df2.Condition == 'Noise']
    GLMresN = GLMresN.reset_index(drop=True)
    GLMresS['ch_name'] = df2['ch_name'][df2.Condition == 'Speech']
    GLMresS['Speech'] = df2['p_value'][df2.Condition == 'Speech']
    GLMresS = GLMresS.reset_index(drop=True)

    GLMres = pd.merge(GLMresN,GLMresS, on = 'ch_name')

    m = len(GLMres)
    FDRarray = np.array(range(1,m+1))
    FDRalpha = 0.2
    

    FDRarray = FDRalpha * FDRarray / m

    for jj,cond in enumerate(['Noise','Speech']):
        GLMres = GLMres.sort_values(by=cond, ascending=True)
        GLMres[cond] = GLMres[cond] < FDRarray
        maxpos = GLMres[cond][::-1].idxmax()
        minpos = GLMres[cond][::-1].idxmin()
        if maxpos == minpos:
            continue
        maxloc = GLMres.index.get_loc(maxpos)
        GLMres[cond].iloc[0:maxloc + 1] = True



    GLMres = GLMres.sort_index()

    #mne.viz.plot_events(events, event_id=event_dict, sfreq=raw_haemo.info['sfreq'],show=True)


    return GLMres

In [ ]:
def fullanalysis(SubjectID,VisitID):

    idxstring = np.array([
        [4, 7],   # For Subject 1
        [9, 7],   # For Subject 2
        [13, 11], # For Subject 3
        [5, 5],   # For Subject 4
        [1, 1],   # For Subject 5
        [1, 1],   # For Subject 6
        [1, 1],   # For Subject 7
        [1, 1],   # For Subject 8
        [1, 1],   # For Subject 9
        [1, 1],   # For Subject 10
        [1, 1],   # For Subject 11
        [1, 1],   # For Subject 12
        [1, 1],   # For Subject 13
        [1, 1],   # For Subject 14
        [1, 1],   # For Subject 15
        [1, 1],   # For Subject 16
        [1, 1],   # For Subject 17
        [1, 1],   # For Subject 18
        [1, 1],   # For Subject 19
        [1, 1],   # For Subject 20
        [1, 1],   # For Subject 21
        [1, 1],   # For Subject 22
        [1, 1],   # For Subject 23
        [1, 1]    # For Subject 24
    ])


    BadEpochList = np.empty((24,2,13),dtype=int)
    BadEpochList[1-1][1-1][0:4]=[2-1, 18-1, 37-1, 54-1]
    BadEpochList[1-1][2-1][0:7]=[8-1, 15-1, 16-1, 17-1, 28-1, 41-1, 60-1]
    BadEpochList[2-1][1-1][0:9]=[3-1, 5-1, 9-1, 16-1, 30-1, 37-1, 38-1, 51-1, 53-1]
    BadEpochList[2-1][2-1][0:7]=[7-1, 8-1, 10-1, 12-1, 51-1, 56-1, 57-1]
    BadEpochList[3-1][1-1][0:13]=[5-1, 8-1, 9-1, 16-1, 18-1, 19-1, 26-1, 33-1, 46-1, 47-1, 49-1, 57-1, 58-1]
    BadEpochList[3-1][2-1][0:11]=[1-1, 3-1, 7-1, 13-1, 16-1, 21-1, 35-1, 46-1, 54-1, 57-1, 60-1]
    BadEpochList[4-1][1-1][0:5]=[7-1, 8-1, 20-1, 41-1, 44-1]
    BadEpochList[4-1][2-1][0:5]=[33-1, 36-1, 38-1, 40-1, 56-1]
    BadEpochList[5-1][1-1][0:1]=[60-1]
    BadEpochList[5-1][2-1][0:1]=[60-1]
    BadEpochList[6-1][1-1][0:1]=[60-1]
    BadEpochList[6-1][2-1][0:1]=[60-1]
    BadEpochList[7-1][1-1][0:1]=[60-1]
    BadEpochList[7-1][2-1][0:1]=[60-1]
    BadEpochList[8-1][1-1][0:1]=[60-1]
    BadEpochList[8-1][2-1][0:1]=[60-1]
    BadEpochList[9-1][1-1][0:1]=[60-1]
    BadEpochList[9-1][2-1][0:1]=[60-1]
    BadEpochList[10-1][1-1][0:1]=[60-1]
    BadEpochList[10-1][2-1][0:1]=[60-1]
    BadEpochList[11-1][1-1][0:1]=[60-1]
    BadEpochList[11-1][2-1][0:1]=[60-1]
    BadEpochList[12-1][1-1][0:1]=[60-1]
    BadEpochList[12-1][2-1][0:1]=[60-1]
    BadEpochList[13-1][1-1][0:1]=[60-1]
    BadEpochList[13-1][2-1][0:1]=[60-1]
    BadEpochList[14-1][1-1][0:1]=[60-1]
    BadEpochList[14-1][2-1][0:1]=[60-1]
    BadEpochList[15-1][1-1][0:1]=[60-1]
    BadEpochList[15-1][2-1][0:1]=[60-1]
    BadEpochList[16-1][1-1][0:1]=[60-1]
    BadEpochList[16-1][2-1][0:1]=[60-1]
    BadEpochList[17-1][1-1][0:1]=[60-1]
    BadEpochList[17-1][2-1][0:1]=[60-1]
    BadEpochList[18-1][1-1][0:1]=[60-1]
    BadEpochList[18-1][2-1][0:1]=[60-1]
    BadEpochList[19-1][1-1][0:1]=[60-1]
    BadEpochList[19-1][2-1][0:1]=[60-1]
    BadEpochList[20-1][1-1][0:1]=[60-1]
    BadEpochList[20-1][2-1][0:1]=[60-1]
    BadEpochList[21-1][1-1][0:1]=[60-1]
    BadEpochList[21-1][2-1][0:1]=[60-1]
    BadEpochList[22-1][1-1][0:1]=[60-1]
    BadEpochList[22-1][2-1][0:1]=[60-1]
    BadEpochList[23-1][1-1][0:1]=[60-1]
    BadEpochList[23-1][2-1][0:1]=[60-1]
    BadEpochList[24-1][1-1][0:1]=[60-1]
    BadEpochList[24-1][2-1][0:1]=[60-1]


    #fnirs_data_folder =(r'C:\Users\Andy\OneDrive - Danmarks Tekniske Universitet\Data\Recordings')
    # New base path
    fnirs_data_folder = r'C:\Users\sarab\Desktop\THESIS\Tutorial MNE\subjects'

    # Build subject and session names based on ID
    subject = f"sub-{SubjectID:02d}"
    session = f"ses-{VisitID:02d}"
    nirs_folder = 'nirs'

    fnirs_cw_amplitude_dir = op.join(fnirs_data_folder, subject, session, nirs_folder)

    raw_intensity = mne.io.read_raw_nirx(fnirs_cw_amplitude_dir)
    raw_intensity.load_data()
    #raw_intensity.info['bads'].extend(['S3_D2 785', 'S3_D2 830'])

    # annotation names
    raw_intensity.annotations.rename({'1.0': 'Control',
                                    '2.0': 'Noise',
                                    '3.0': 'Speech',
                                    '4.0': 'Xstart',
                                    '5.0': 'Xend'})

    # annotation durations
    raw_intensity.annotations.set_durations({'Control' : 5, 'Noise' : 5, 'Speech' : 5.25})

    # Break events
    Breaks, event_dict = mne.events_from_annotations(raw_intensity,{'Xend': 4, 'Xstart': 5})
    # All events
    AllEvents, event_dict = mne.events_from_annotations(raw_intensity)

    # Converting from index to time
    Breaks = Breaks[:,0]/raw_intensity.info['sfreq']
    LastEvent = AllEvents[-1,0]/raw_intensity.info['sfreq']


    # constructing block for each block in experiment
    cropped_intensity = raw_intensity.copy().crop(Breaks[0],Breaks[1])
    block2 = raw_intensity.copy().crop(Breaks[2],Breaks[3])
    block3 = raw_intensity.copy().crop(Breaks[4],Breaks[5])
    block4 = raw_intensity.copy().crop(Breaks[6], LastEvent+15.25)

    # Combining all blocks
    cropped_intensity.append([block2,block3,block4])
    cropped_od = mne.preprocessing.nirs.optical_density(cropped_intensity)
    cropped_corrected_od = temporal_derivative_distribution_repair(cropped_od)


    sci = mne.preprocessing.nirs.scalp_coupling_index(cropped_corrected_od)
    """ fig, ax = plt.subplots()
    ax.hist(sci)
    ax.set(xlabel='Scalp Coupling Index', ylabel='Count', xlim=[0, 1])
    """

    
    # removing break annotations
    unwanted = np.nonzero(raw_intensity.annotations.description == 'Xstart')
    raw_intensity.annotations.delete(unwanted)
    unwanted = np.nonzero(raw_intensity.annotations.description == 'Xend')
    raw_intensity.annotations.delete(unwanted)

    #raw_intensity.annotations.delete(BadEpochList[SubjectID-1][VisitID-1][0:idxstring[SubjectID-1,VisitID-1]])


    raw_od = mne.preprocessing.nirs.optical_density(raw_intensity)


    corrected_od = temporal_derivative_distribution_repair(raw_od)


    corrected_od.info['bads'] = list(compress(cropped_corrected_od.ch_names, sci < 0.7))
    #corrected_od.info['bads'].extend(['S3_D2 785', 'S3_D2 830'])
    
    raw_haemo = mne.preprocessing.nirs.beer_lambert_law(corrected_od, ppf=0.1)
    raw_haemo.plot(n_channels=len(raw_haemo.ch_names),
                duration=500, show_scrollbars=False,show=False)


    """ fig = raw_haemo.plot_psd(average=True,show=False)
    fig.suptitle('Before filtering', weight='bold', size='x-large')
    fig.subplots_adjust(top=0.88) """


    raw_haemo = raw_haemo.filter(l_freq = None, h_freq = 0.2,  
                                 method="iir", iir_params =dict(order=5, ftype='butter'))
     #high-pass
    raw_haemo= raw_haemo.filter(l_freq =  0.005, h_freq = None, method="iir", iir_params =dict(order=5, ftype='butter'))
    
    """ fig = raw_haemo.plot_psd(average=True,show=False)
    fig.suptitle('After filtering', weight='bold', size='x-large')
    fig.subplots_adjust(top=0.88) """


    raw_haemo.plot(n_channels=len(raw_haemo.ch_names),
                duration=500, show_scrollbars=False,show=False)


    WAAres,WaaGrpDF, ROIs = weighted_averaging_analysis(raw_haemo)

    GLMres = GLM_analysis(raw_haemo)
    return WAAres, WaaGrpDF, ROIs, GLMres

In [5]:
WAAres = {}    
WaaRoiGrp = {}
GLMres = {}

roilist = ['front', 'left', 'back', 'right']


# run analysis and save results in csv file
# Range (1,5) is actually = 1,2,3,4

 # whatever subjects you want


for SubjectID in range(5,6):
    if SubjectID == 9:
        continue
    for VisitID in range (1,2):
        dfwaa, WaaGrpDF, ROIs, dfglm = fullanalysis(SubjectID,VisitID)
        dfwaa.to_csv('WAA' + str(SubjectID) + str(VisitID) + '.csv', index=False)
        dfglm.to_csv('GLM' + str(SubjectID) + str(VisitID) + '.csv', index=False)
        WaaRoiGrp[(SubjectID-1)*2+VisitID] = WaaGrpDF

# For group level waa plot,

# Epochs are probably not saved correctly from the analysis, the 95% confidence interval does not work here.
# Find a better way to store the epochs.
condlist =  [['ControlHbO','ControlHbR'],['NoiseHbO','NoiseHbR'],['SpeechHbO','SpeechHbR']]

for test in range (2,9):
    for condidx, cond in enumerate(condlist):
        WaaRoiGrp[1][cond[0]]._data = WaaRoiGrp[1][cond[0]]._data + WaaRoiGrp[test][cond[0]]._data
        WaaRoiGrp[1][cond[1]]._data = WaaRoiGrp[1][cond[1]]._data + WaaRoiGrp[test][cond[1]]._data


for ii in range(len(condlist)):
    WaaRoiGrp[1][condlist[ii][0]]._data = WaaRoiGrp[1][condlist[ii][0]]._data /8
    WaaRoiGrp[1][condlist[ii][1]]._data = WaaRoiGrp[1][condlist[ii][1]]._data /8

fig, axes = plt.subplots(nrows=len(ROIs), ncols=len(condlist),figsize=(14, 10))
titlestr = ['Control','Noise','Speech']
for ridx, roi in enumerate(ROIs):
    for cidx, event in enumerate(condlist): 
        plot_compare_evokeds([WaaRoiGrp[1][event[0]],WaaRoiGrp[1][event[1]]], picks=ROIs[roi], combine="mean", title='', 
            axes=axes[ridx,cidx], show=False,colors=['r','b'], ylim=dict(hbo=[-0.2, 0.4],hbr=[-0.2, 0.4]), legend=False, show_sensors=True, ci=0.95)
        
        axes[0,cidx].set_title(f'{titlestr[cidx]}')
        axes[ridx, 0].set_ylabel(f"{roi}\n μM")

axes[0, 0].legend(["Oxyhaemoglobin", "Deoxyhaemoglobin"],loc=2)



print('dont stop debugging')

Loading C:\Users\sarab\Desktop\THESIS\Tutorial MNE\subjects\sub-05\ses-01\nirs
Reading 0 ... 8666  =      0.000 ...  1663.872 secs...


C:\Users\sarab\AppData\Local\Temp\ipykernel_38120\1544337469.py:93: RuntimeWarning: Extraction of measurement date from NIRX file failed. This can be caused by files saved in certain locales (currently only ['en_US.utf8', 'de_DE', 'fr_FR', 'it_IT'] supported). Please report this as a github issue. The date is being set to January 1st, 2000, instead of '"27. jun 2023""16:28:00.309"'.
  raw_intensity = mne.io.read_raw_nirx(fnirs_cw_amplitude_dir)


Used Annotations descriptions: [np.str_('Xend'), np.str_('Xstart')]
Used Annotations descriptions: [np.str_('Control'), np.str_('Noise'), np.str_('Speech'), np.str_('Xend'), np.str_('Xstart')]
Using qt as 2D backend.
Filtering raw data in 1 contiguous segment
Setting up low-pass filter at 0.2 Hz

IIR filter parameters
---------------------
Butterworth lowpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 10 (effective, after forward-backward)
- Cutoff at 0.20 Hz: -6.02 dB

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.005 Hz

IIR filter parameters
---------------------
Butterworth highpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 10 (effective, after forward-backward)
- Cutoff at 0.01 Hz: -6.02 dB

Used Annotations descriptions: [np.str_('Control'), np.str_('Noise'), np.str_('Speech')]
Not setting metadata
60 matching events found
Setting baseline interval to [-4.992000319488021, 0.0] s
Apply

: 